In [2]:
from google_maps_scraper import google_maps_scraper

In [13]:
wizard = google_maps_scraper(['Beach Park', 'Petverso Café'])

Iniciando Scraper...


In [14]:
dados = wizard.coletar(5)

Horário de início: 2023-02-27 21:34:30.912689
Beach Park
qtd comentarios na página (min = 5): 5
Petverso Café
qtd comentarios na página (min = 5): 7
Coleta finalizada! Fechando navegador... (Horário: 2023-02-27 21:35:12.031818)


Horário de fim: 2023-02-27 21:35:14.171001


In [15]:
print(dados['notas'])

  nome_do_estabelecimento nota  qtd_comentarios   timestamp_coleta
0              Beach Park  4,5              759  27/02/23 21:34:32
1           Petverso Café  4,4               39  27/02/23 21:35:04


In [16]:
print(dados['palavras'])

   nome_do_estabelecimento    palavra_chave mencoes   timestamp_coleta
0               Beach Park            preço      38  27/02/23 21:34:32
1               Beach Park         diversão      13  27/02/23 21:34:32
2               Beach Park  parque aquático       9  27/02/23 21:34:32
3               Beach Park              mar       9  27/02/23 21:34:32
4               Beach Park          salgado       7  27/02/23 21:34:32
5               Beach Park       brinquedos       6  27/02/23 21:34:32
6               Beach Park         crianças       6  27/02/23 21:34:32
7               Beach Park          valores       5  27/02/23 21:34:32
8               Beach Park          serviço       5  27/02/23 21:34:32
9               Beach Park           férias       3  27/02/23 21:34:32
10           Petverso Café         ambiente      11  27/02/23 21:35:04
11           Petverso Café         gatinhos       8  27/02/23 21:35:04
12           Petverso Café            preço       4  27/02/23 21:35:04
13    

In [20]:
print(dados['comentarios'])

  nome_do_estabelecimento                                         comentario  \
0              Beach Park  Espaço incrível. Conforto nota 10.\nSuper priv...   
1              Beach Park          (Tradução do Google) OK\n\n(Original)\nOk   
2              Beach Park  Maravilhoso! Ambiente a beira da praia, lounge...   
3              Beach Park  Preço salgado e a seleta de frutos do mar R$40...   
4              Beach Park  Excelente opção, atendimento muito bom e tudo ...   
0           Petverso Café  Ambiente massa,comidas maravilhosas e lindos a...   
1           Petverso Café                                Proposta inovadora.   
2           Petverso Café  Levei minha filha que ficou encantada com os g...   
3           Petverso Café              Excelente 👏👏👏. Voltarei e recomendo …   
4           Petverso Café  Fui nesse local para conhecer os gatinhos e ac...   
5           Petverso Café  Ótima comida, e depois de comer você ainda pod...   
6           Petverso Café  Fomos conhece